<a href="https://colab.research.google.com/github/cakarsubasi/models/blob/master/training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow-gpu
!pip install tensorflow
!pip install tf_slim
!pip install pycocotools

     |████████████████████████████████| 497.5 MB 26 kB/s 
     |████████████████████████████████| 462 kB 48.3 MB/s 
     |████████████████████████████████| 352 kB 4.2 MB/s 


In [2]:
import os
import pathlib
import python


if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/cakarsubasi/models.git

Cloning into 'models'...
remote: Enumerating objects: 3492, done.
remote: Counting objects: 100% (3492/3492), done.
remote: Compressing objects: 100% (2775/2775), done.
remote: Total 3492 (delta 1052), reused 1585 (delta 666), pack-reused 0
Receiving objects: 100% (3492/3492), 44.26 MiB | 11.98 MiB/s, done.
Resolving deltas: 100% (1052/1052), done.


In [3]:
%%bash
cd /content/models/research
protoc object_detection/protos/*.proto --python_out=.

In [ ]:
%%bash 
cd /content/models/research
pip install .

In [5]:
%cd models

/content/models


In [6]:
import tensorflow as tf
from research.object_detection.utils import dataset_util
from research.object_detection.utils import visualization_utils as viz_utils
from research.object_detection.utils import label_map_util
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()


import numpy as np 
import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline

from PIL import Image
from IPython.display import display
#matplotlib.use('TkAgg')
#import tkinter 
#import ast

In [7]:
%%bash
cd /content/models/research
git clone https://github.com/cocodataset/cocoapi.git
cd cocoapi/PythonAPI/
make
cp -r pycocotools /content/models/research
cd ../..
## is this really needed? API should already be installed
cp object_detection/packages/tf2/setup.py .
python -m pip install .
python object_detection/builders/model_builder_tf2_test.py
cd ..

python setup.py build_ext --inplace
running build_ext
cythoning pycocotools/_mask.pyx to pycocotools/_mask.c
building 'pycocotools._mask' extension
creating build
creating build/common
creating build/temp.linux-x86_64-3.7
creating build/temp.linux-x86_64-3.7/pycocotools
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fdebug-prefix-map=/build/python3.7-pX47U3/python3.7-3.7.12=. -fstack-protector-strong -Wformat -Werror=format-security -g -fdebug-prefix-map=/build/python3.7-pX47U3/python3.7-3.7.12=. -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/local/lib/python3.7/dist-packages/numpy/core/include -I../common -I/usr/include/python3.7m -c ../common/maskApi.c -o build/temp.linux-x86_64-3.7/../common/maskApi.o -Wno-cpp -Wno-unused-function -std=c99
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fdebug-prefix-map=/build/python3.7-pX47U3/

Cloning into 'cocoapi'...
../common/maskApi.c: In function ‘rleDecode’:
../common/maskApi.c:46:7: warning: this ‘for’ clause does not guard... [-Wmisleading-indentation]
       for( k=0; k<R[i].cnts[j]; k++ ) *(M++)=v; v=!v; }}
       ^~~
../common/maskApi.c:46:49: note: ...this statement, but the latter is misleadingly indented as if it were guarded by the ‘for’
       for( k=0; k<R[i].cnts[j]; k++ ) *(M++)=v; v=!v; }}
                                                 ^
../common/maskApi.c: In function ‘rleFrPoly’:
../common/maskApi.c:166:3: warning: this ‘for’ clause does not guard... [-Wmisleading-indentation]
   for(j=0; j<k; j++) x[j]=(int)(scale*xy[j*2+0]+.5); x[k]=x[0];
   ^~~
../common/maskApi.c:166:54: note: ...this statement, but the latter is misleadingly indented as if it were guarded by the ‘for’
   for(j=0; j<k; j++) x[j]=(int)(scale*xy[j*2+0]+.5); x[k]=x[0];
                                                      ^
../common/maskApi.c:167:3: warning: this ‘for’ clause does 

In [8]:
record_train = "data/muscima/training.record"
record_validation = "data/muscima/validation.record"
record_test = "data/muscima/test.record"

class_map = "mapping_all_classes.txt"

In [ ]:
def generate_omr_dataset_from_tfrecord(filename: str):
    dataset = tf.data.TFRecordDataset(filename)

    dataset = dataset.map(
        parse_omr_element
        )

    return dataset

    
def parse_omr_element(element):

    data = {
    'image/height': tf.io.FixedLenFeature([], tf.int64),
    'image/width': tf.io.FixedLenFeature([], tf.int64),
    'image/filename': tf.io.FixedLenFeature([], tf.string),
    'image/source_id': tf.io.FixedLenFeature([], tf.string),
    #'image/key/sha256':tf.io.FixedLenFeature([], tf.string),
    'image/encoded': tf.io.FixedLenFeature([], tf.string),
    #'image/encoded': tf.io.decode_png([], channels=0),
    'image/format': tf.io.FixedLenFeature([], tf.string),
    'image/object/bbox/xmin': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'image/object/bbox/xmax': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'image/object/bbox/ymin': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'image/object/bbox/ymax': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'image/object/class/text': tf.io.FixedLenSequenceFeature([], tf.string, allow_missing=True),
    'image/object/class/label': tf.io.FixedLenSequenceFeature([], tf.int64, allow_missing=True),
    }

    example = tf.io.parse_single_example(element, data)

    example['image/decoded'] = tf.io.decode_image(example.get('image/encoded'))

    example_np_xmin = example['image/object/bbox/xmin']
    example_np_xmax = example['image/object/bbox/xmax']
    example_np_ymin = example['image/object/bbox/ymin']
    example_np_ymax = example['image/object/bbox/ymax']
    
    example_np_bbox = tf.stack((example_np_ymin, example_np_xmin, example_np_ymax, example_np_xmax))

    example['image/bbox'] = tf.transpose(example_np_bbox)

    # get rid of irrelevant objects
    example.pop('image/encoded')
    example.pop('image/format')
    example.pop('image/object/bbox/xmin')
    example.pop('image/object/bbox/ymin')
    example.pop('image/object/bbox/xmax')
    example.pop('image/object/bbox/ymax')

    return example


def plot_detections_from_dataset_element(element):
  dummy_scores = np.repeat(np.array([1.0], dtype=np.float32), element['image/object/class/label'].shape[0])
  label_map_dict = label_map_util.get_label_map_dict(class_map)
  image_np = element['image/decoded'].numpy().copy()
  image_np = np.repeat(image_np, 3, 2)
  boxes_np = element['image/bbox'].numpy()
  classes_np = element['image/object/class/label'].numpy()
  viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np,
                boxes=boxes_np,
                classes=classes_np,
                scores=dummy_scores,
                category_index=label_map_dict,
                use_normalized_coordinates=True,
                min_score_thresh = 0.8,
                max_boxes_to_draw=dummy_scores.shape[0]
  )
  return image_np

In [ ]:
training_dataset = generate_omr_dataset_from_tfrecord(record_train)
validation_dataset = generate_omr_dataset_from_tfrecord(record_validation)
test_dataset = generate_omr_dataset_from_tfrecord(record_test)
print(training_dataset)

<MapDataset element_spec={'image/filename': TensorSpec(shape=(), dtype=tf.string, name=None), 'image/height': TensorSpec(shape=(), dtype=tf.int64, name=None), 'image/object/class/label': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'image/object/class/text': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'image/source_id': TensorSpec(shape=(), dtype=tf.string, name=None), 'image/width': TensorSpec(shape=(), dtype=tf.int64, name=None), 'image/decoded': TensorSpec(shape=<unknown>, dtype=tf.uint8, name=None), 'image/bbox': TensorSpec(shape=(None, 4), dtype=tf.float32, name=None)}>


In [9]:
# convert the label map to a suitable format

label_map_dict_raw = label_map_util.get_label_map_dict(class_map)

label_map_dict = {}

for key in label_map_dict_raw.keys():
    value = {
        'id': label_map_dict_raw.get(key),
        'name': key
    }
    label_map_dict[label_map_dict_raw.get(key)] = value

In [ ]:
def plot_detections_from_dataset_element(element):
  dummy_scores = np.repeat(np.array([1.0], dtype=np.float32), element['image/object/class/label'].shape[0])
  image_np = element['image/decoded'].numpy().copy()
  image_np = np.repeat(image_np, 3, 2)
  boxes_np = element['image/bbox'].numpy()
  classes_np = element['image/object/class/label'].numpy()
  viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np,
                boxes=boxes_np,
                classes=classes_np,
                scores=dummy_scores,
                category_index=label_map_dict,
                use_normalized_coordinates=True,
                min_score_thresh = 0.8,
                max_boxes_to_draw=dummy_scores.shape[0]
  )
  plt.figure(figsize=(30,15))
  plt.imshow(image_np)
  plt.show()



In [ ]:
# Visualize an element from the training dataset

sample = training_dataset.take(1)
td_element = sample.get_single_element()

plot_detections_from_dataset_element(td_element)

In [ ]:
def load_model(model_name):
  base_url = 'http://download.tensorflow.org/models/object_detection/'
  model_file = model_name + '.tar.gz'
  model_dir = tf.keras.utils.get_file(
    fname=model_name, 
    origin=base_url + model_file,
    untar=True)

  model_dir = pathlib.Path(model_dir)/"saved_model"

  model = tf.saved_model.load(str(model_dir))

  return model

In [13]:
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz
!tar -xf ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz
#-r to do recursive moves
!mv /content/models/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/checkpoint /content/models/research/object_detection/test_data

--2022-03-14 04:49:48--  http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 142.250.157.128, 2404:6800:4008:c13::80
Connecting to download.tensorflow.org (download.tensorflow.org)|142.250.157.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 244817203 (233M) [application/x-tar]
Saving to: ‘ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz.3’

ssd_resnet50_v1_fpn 100%[===================>] 233.48M   207MB/s    in 1.1s    

2022-03-14 04:49:49 (207 MB/s) - ‘ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz.3’ saved [244817203/244817203]



In [25]:
tf.keras.backend.clear_session()

print('Building model and restoring weights for fine-tuning...', flush=True)
num_classes = 110
pipeline_config = '/content/models/research/object_detection/configs/tf2/elec491-ssd_mobilenet_v1_fpn_640x640_coco17_tpu-8.config'
checkpoint_path = '/content/models/research/object_detection/test_data/checkpoint/ckpt-0.index'


Building model and restoring weights for fine-tuning...


In [44]:
import python

ModuleNotFoundError: ignored

In [26]:
from object_detection.utils import config_util
from object_detection.builders import model_builder
configs = config_util.get_configs_from_pipeline_file(pipeline_config)
model_config = configs['model']
model_config.ssd.num_classes = num_classes
model_config.ssd.freeze_batchnorm = True
detection_model = model_builder.build(
      model_config=model_config, is_training=True)


In [33]:
PIPELINE_CONFIG_PATH= '/content/models/research/object_detection/configs/tf2/elec491-ssd_mobilenet_v1_fpn_640x640_coco17_tpu-8.config'
MODEL_DIR= '/content/models/research/object_detection/models/ssd_resnet_v1_fpn_feature_extractor.py'



In [49]:
python /content/models/research/object_detection/model_main_tf2.py --model_dir=MODEL_DIR --pipeline_config_path=PIPELINE_CONFIG_PATH

SyntaxError: ignored